In [ ]:
# Evita falhas de falta de Lib's
!pip install -r requirements.txt --upgrade --quiet

In [ ]:
#Importações
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#Carrega dataFrames
# Método que carrega a tabela de entregas
def carregar_dados():
    # Le o excel
    df = pd.read_excel('Datalog.xlsx', sheet_name='Entregas', parse_dates=['Data'])
    # Evita um erro e um Warning
    df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y', errors='coerce')
    # Retorna o Data Frame
    return df
    
# Método que carrega a tabela de Veiculos
def carrega_dados_veiculos():
    # Le o excel
    df = pd.read_excel('Datalog.xlsx', sheet_name='Veiculos')
    # Retorna o Data Frame
    return df

# Método que carrega a tabela de Demandas
def carrega_dados_demandas():
    # Le o excel
    df = pd.read_excel('Datalog.xlsx', sheet_name='Demandas', parse_dates=['Data'])
    # Evita um erro e um Warning
    df['Data'] = pd.to_datetime(df['Data'], errors='coerce') 
    # Retorna o Data Frame
    return df

# Método que carrega a tabela de Financeiro
def carrega_dados_financeiro():
    # Le o excel
    df = pd.read_excel('Datalog.xlsx', sheet_name='Financeiros')
    # Retorna o Data Frame
    return df

# Carrega data frame e coloca em uma váriavel
df_entregas = carregar_dados()
df_veiculos = carrega_dados_veiculos()
df_demanda = carrega_dados_demandas()
df_financeiro = carrega_dados_financeiro()

In [ ]:
# Análise de entregas por destino
# Método responsável por exibir o gráfico de análise de entregas p/ estado(Destino)
def analise_entregas_estado(df, sistema = False):
    # Conta a quantidade de destinos
    counts = df['Destino'].value_counts()
    #Colore o Dashboard
    counts.plot(kind='bar', color='lightgreen')
    # Coloca o titulo no gráfico
    plt.title('Entregas por Destino')
    # Passa o titulo do eixo Y
    plt.ylabel('Contagem')
    # ajeita o eixo do gráfico
    plt.xticks(rotation=30, ha='right')
    # Ajusta o layout para evitar sobreposição
    plt.tight_layout()
    # Exibe o gráfico
    plt.show()
        
analise_entregas_estado(df_entregas)

### Análise de Status das entregas (A caminho, Entregue, Atrasado)

In [ ]:
# Um gráfico de pizza que exibe o status de entrega
def analise_status_entregas(df, sistema = False):
    # Pega o tamanho total de entregas
    total_entregas = len(df)
    # Conta a quantidade de valores do "Status entrega"
    estados = df['Status da Entrega'].value_counts()
    # Exibe a porcentagem de entregas p/ estado
    porcentagens = (estados / total_entregas * 100).round(2)
    # Cria uma figura e um ax
    fig1, ax1 = plt.subplots()
    # Configura o ax
    ax1.pie(porcentagens, labels=porcentagens.index, autopct='%1.1f%%', startangle=90)
    # Garante que o gráfico de pizza seja circular(segundo a documentação :D)
    ax1.axis('equal')
    # Exibe o gráfico
    plt.show()
        
analise_status_entregas(df_entregas)

## Quantidade de entregas P/ Data

In [ ]:
def analise_entregas_por_data(df, sistema = False):
    # Passa o campo data de entrega pro Data frame
    df['Data'] = pd.to_datetime(df['Data'])  # Data estimada de entrega
    # Faz um "Count" diferente, pois se trata de data, e organiza o indicde por isso
    counts = df['Data'].dt.date.value_counts().sort_index()
    # Escolhe o tipo de grafico
    counts.plot(kind='line', marker='o')
    # Passa o titulo do grafico
    plt.title('Entregas Estimadas por Data')
    # Passa o testo da horizontal
    plt.xlabel('Data Estimada')
    # Passa o texto na vertical
    plt.ylabel('Número de Entregas')
    # Ajusta os nomes
    plt.xticks(rotation=45)
    # Ajusta o layout para evitar sobreposição
    plt.tight_layout()
    # Exibe o gráfico
    plt.show()
    # Verifica se o sistema está sendo usado
# Exibe o grafico
analise_entregas_por_data(df_entregas)

## Atrasadas x No prazo

In [ ]:
def analise_entregas_atrasadas(df, sistema = False):
    # Passa o campo data de entrega pro Data frame
    df['Data'] = pd.to_datetime(df['Data'])  
    # Campo data de entrega + Tempo de atraso
    df['Data de Entrega'] = df['Data'] + pd.to_timedelta(df['Tempo'], unit='D')
    # Exibe as que o tempo de atraso, for maior que a data estimada de entrega
    df['Atrasada'] = df['Data de Entrega'] > df['Data']
    # Conta a quantidade de atrasadas
    counts = df['Atrasada'].value_counts()
    # Passa pros indices
    counts.index = ['No Prazo', 'Atrasada']
    # Estiliza o Gráfico
    counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=['lightblue', 'salmon'])
    # Passa o titulo do grafico
    plt.title('Entregas Atrasadas vs No Prazo')
    # Titulo 2
    plt.ylabel('')
    # Ajusta o layout para evitar sobreposição
    plt.tight_layout()
    # Exibe o gráfico
    plt.show()

    
analise_entregas_atrasadas(df_entregas)

In [ ]:
def analise_motivos_atrasos(df, sistema=False):
    # Passa o campo data de entrega pro Data frame
    df['Data'] = pd.to_datetime(df['Data']) 
    # Campo data de entrega + Tempo de atraso
    df['Data de Entrega'] = df['Data'] + pd.to_timedelta(df['Tempo'], unit='D')
    # Passa pra um DF o que for maior que a data estimada de entrega
    df['Atrasada'] = df['Data de Entrega'] > df['Data']
    # Confirma se tem atrasadas
    df_atrasadas = df[df['Atrasada'] == True]
    # Verifica se não tem atrasadas
    if df_atrasadas.empty:
        print("Nenhuma entrega atrasada encontrada.")
    else:
        # Se tiver
        # Passa os motivos pra uma váriavel
        motivos = df_atrasadas['Incidente de Trânsito'].value_counts()
        # Deixa o gráfico decorado
        motivos.plot(kind='bar', color='salmon', edgecolor='black')
        # Passa o titulo do grafico
        plt.title('Motivos dos Atrasos nas Entregas')
        # Passa o testo da horizontal
        plt.xlabel('Motivo')
        # Passa o texto na vertical
        plt.ylabel('Quantidade')
        # Deixa mais bonito os nomes
        plt.xticks(rotation=45, ha='right')
        # Ajusta o layout para evitar sobreposição
        plt.tight_layout()
        # Exibe o gráfico
        plt.show()
        # Verifica se o sistema está sendo usado
        
analise_motivos_atrasos(df_entregas)

In [ ]:
def analise_tempos_entrega(df, sistema=False):
    # Verificar se o código está sendo chamado pelo sistema
    if(sistema == True):
        # Limpa a tela
        clear_output(wait=True)
    # Seta campo data
    df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
    # Campo data de entrega + Prazo de entrega
    df['Data de Entrega Original'] = df['Data'] + pd.to_timedelta(df['Prazo de Entrega (dias)'], unit='D')
    # Campo data de entrega + Tempo de atraso
    df['Data de Entrega Com Atraso'] = df['Data de Entrega Original'] + pd.to_timedelta(df['Tempo'], unit='D')
    
    # Passa pra um DF as que o tempo de atraso, for maior que a data estimada de entrega
    df['Atrasada'] = df['Data de Entrega Com Atraso'] > df['Data de Entrega Original']
    
    # Filtra o DataFrame para obter apenas as entregas atrasadas
    df_atrasadas = df[df['Atrasada'] == True]
    
    # Verifica se não há entregas atrasadas
    if df_atrasadas.empty:
        print("Nenhuma entrega atrasada encontrada.")
    else:
        # Se houver entregas atrasadas, seleciona as colunas relevantes para análise
        tempos = df_atrasadas[['Centro de Distribuição', 'Destino', 'Data de Entrega Original',
                               'Data de Entrega Com Atraso', 'Tempo', 'Prazo de Entrega (dias)',
                               'Incidente de Trânsito']].copy()
        
        fig, ax = plt.subplots()
        ax.axis('off')

        col_labels = ['Centro de Distribuição', 'Destino', 'Data de Entrega\nOriginal',
                      'Data de Entrega\nCom Atraso', 'Atraso(dias)', 'Prazo de Entrega\n(dias)',
                      'Incidente de Trânsito']
         # Cria a tabela com os dados filtrados, rótulos e formatação personalizada
        table = ax.table(
            cellText=tempos.values,
            colLabels=col_labels,
            loc='center',
            cellLoc='center',
            colWidths=[0.2, 0.2, 0.3, 0.3, 0.15, 0.2, 0.2]
        )
        # Define o tamanho da fonte 
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        # Define tamanho da escala
        table.scale(1.2, 1.5)
        # Ajusta o tamanho da figura de acordo com o conteúdo da tabela
        bbox = table.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.set_size_inches(bbox.width, bbox.height)
         # Exibe a tabela na tela
        plt.show()
        
analise_tempos_entrega(df_entregas)

# ANÁLISE DEMANDAS
> Pico Sazonal

> Análise Pedidos Reais vs Esperado

> Análise Demanda de produto

In [ ]:
def analise_real_vs_previsto(df, sistema = False):
    # Faz uma copia pra alterar o dataframe
    df_temp = df.copy()
    #Escolhe o mes e ano
    df_temp['Ano-Mês'] = df_temp['Data'].dt.to_period('M').astype(str)
    # tenta agrupar o mes e ano
    agrupado = df_temp.groupby('Ano-Mês')[['Volume de Vendas', 'Previsão de Demanda']].sum().sort_index()
    # Seleciona o tamanho da figura
    plt.figure(figsize=(14, 6))
    # Escolhe indices pra serem a previsão de vedas, e volume real
    plt.plot(agrupado.index, agrupado['Volume de Vendas'], label='Real', marker='o', color='royalblue')
    plt.plot(agrupado.index, agrupado['Previsão de Demanda'], label='Previsto', marker='x', color='orange')
    # Titulo do gráfico
    plt.title('Comparativo Real vs. Previsto da Demanda')
    # Horizontal
    plt.xlabel('Ano-Mês')
    #Vertical
    plt.ylabel('Volume')
    # Leganda
    plt.legend()
    # Ajusta os nomes
    plt.xticks(rotation=45)
    # Ajusta o layout para evitar sobreposição
    plt.tight_layout()
    # Exbibe o dashboard
    plt.show()
    # Verifica se o sistema está sendo usado
analise_real_vs_previsto(df_demanda)

In [ ]:
def analise_demanda_produto(df, sistema = False):
    # Passa a variavél de data
    df['Data'] = pd.to_datetime(df['Data'])
    # Pega por mes
    df['Ano-Mês'] = df['Data'].dt.to_period('M').astype(str)
    # Organiza o df
    df_grouped = df.groupby(['Ano-Mês', 'Produto'])['Volume de Vendas'].sum().unstack(fill_value=0)
    # Organiza o DataFrame
    df_grouped = df_grouped.sort_index()
    
    plt.figure(figsize=(14, 7))
    # Loop pra pegar as colunas pra passar como linhas no gráfico
    for produto in df_grouped.columns:
        plt.plot(df_grouped.index, df_grouped[produto], label=produto, marker='o')
    # Horizontal
    plt.xlabel('Ano-Mês')
    # Vertical
    plt.ylabel('Volume de Vendas')
    # Ajusta os nomes
    plt.xticks(rotation=45)
    #Legenda
    plt.legend(title='Produto', bbox_to_anchor=(1.05, 1), loc='upper left')
    # Selecioona o tipo de linhas
    plt.grid(True, linestyle='--', alpha=0.5)
    # Ajusta o layout para evitar sobreposição
    plt.tight_layout()
    # Exbibe o dashboard
    plt.show()
    
analise_demanda_produto(df_demanda)

# VEÍCULOS 

> Caminhões por Centro de Distribuição

> Capacidade por Centro de Distribuição

> Velocidade Média por Modelo de Caminhão

> Distribuição de Modelos de Caminhões


In [ ]:
def analise_veiculos_disponiveis(df, sistema = False):
    # Verifica se o sistema está sendo usado
    if(sistema == True):
        input("\nPressione Enter para voltar ao menu...")
    # Organiza o DF pela linha especifica
    df_sorted = df.sort_values("Número de Caminhões Disponíveis", ascending=False) 
    # Coloca uma organização especifica
    df_sorted.plot(
        x='Centro de Distribuição',
        y='Número de Caminhões Disponíveis',
        kind='bar',
        color='skyblue',
        legend=False
    )
    # Titulo pro gráfico
    plt.title('Quantidade de Caminhões por Centro de Distribuição')
    # Horizontal
    plt.xlabel('Centro de Distribuição')
    # Verical
    plt.ylabel('Nº de Caminhões')
    # Ajusta os nomes
    plt.xticks(rotation=30, ha='right')
    # Ajusta o layout para evitar sobreposição
    plt.tight_layout()
    # Exibe o gráfico
    plt.show()
    # Verifica se o sistema está sendo usado
    
analise_veiculos_disponiveis(df_veiculos)

In [ ]:

def analise_capacidade_total(df, sistema = False):
    # Nova Coluna pro dataframe
    df["Capacidade Total (Toneladas)"] = df["Número de Caminhões Disponíveis"] * df["Capacidade de Carga (Toneladas)"]
    # Organiza pela Coluna
    df_sorted = df.sort_values("Capacidade Total (Toneladas)", ascending=False)

    # Organiza o gráfico
    df_sorted.plot(
        x='Centro de Distribuição',
        y='Capacidade Total (Toneladas)',
        kind='bar',
        color='orange',
        legend=False
    )
    #Passa o titulo pro gráfico
    plt.title('Capacidade Total de Carga por Centro de Distribuição')
    # Horizontal
    plt.xlabel('Centro de Distribuição')
    # Vertical
    plt.ylabel('Capacidade Total (Toneladas)')
    # Ajusta os nomes
    plt.xticks(rotation=30, ha='right')
    # Ajusta o layout para evitar sobreposição
    plt.tight_layout()
    # Exibe o gráfico
    plt.show()
        
analise_capacidade_total(df_veiculos)

In [ ]:
def analise_velocidade_media(df, sistema = False):
    # Organiza o Data Frame pela coluna especifica
    df_sorted = df.sort_values("Velocidade média (KM/h)", ascending=False)

    # Gera o plot do DF
    df_sorted.plot(
        x='Modelo',
        y='Velocidade média (KM/h)',
        kind='bar',
        color='lightcoral',
        legend=False
    )
    # Titulo pro gráfico
    plt.title('Velocidade Média dos Veículos por Modelo de Caminhão')
    # Horizontal
    plt.xlabel('Modelo')
    # Vertical
    plt.ylabel('Velocidade Média (KM/h)')
    # Ajusta os nomes
    plt.xticks(rotation=30, ha='right')
    # Ajusta o layout para evitar sobreposição
    plt.tight_layout()
    # Exibe o gráfico
    plt.show()
    # Verifica se o sistema está sendo usado
    
analise_velocidade_media(df_veiculos)

In [ ]:
def analise_modelos(df, sistema = False):
    # Faz um group by nos modelos
    soma_por_modelo = df.groupby('Modelo')['Número de Caminhões Disponíveis'].sum()
    # Faz um Plot
    soma_por_modelo.plot(kind='pie', autopct='%1.1f%%', startangle=90)
    # Titulo pro gráfico
    plt.title('Distribuição dos Caminhões por Modelo')
    # Vertical
    plt.ylabel('')
    # Ajusta o layout para evitar sobreposição
    plt.tight_layout()
    # Exibe o gráfico
    plt.show()
    # Verifica se o sistema está sendo usado
    
analise_modelos(df_veiculos)

# Financeiro 

> Custo operacional médio por Centro de distribuição

> Gasto médio Por centro de distribuição(Levando em conta tipo de caminhão utilizado) 






In [ ]:
def analise_custos_operacionais(df_fin, sistema=False):
    # Verifica se o sistema está sendo usado
    if (sistema == True):
        clear_output(wait=True)
    # Valor total do custo operacional
    total = df_fin['Custo Operacional Total (R$/km)'].sum()
    # Grupo By no df
    resumo = df_fin.groupby('Centro de Distribuição')['Custo Operacional Total (R$/km)'] \
                    .sum().reset_index()
    # Custo operacional em %
    resumo['Porcentagem'] = (resumo['Custo Operacional Total (R$/km)'] / total * 100).round(2)
    
    # organiza o plot
    fig, ax = plt.subplots()
    # Formata o gráfico
    ax.pie(resumo['Porcentagem'], labels=resumo['Centro de Distribuição'], autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    # Titulo
    plt.title('Custo Operacional Total por Centro')
    #Exibe o Gráfico
    plt.show()
    # Verifica se o sistema está sendo usado
    
analise_custos_operacionais(df_financeiro)

In [ ]:
def analise_top_distancias(df, sistema=False):
    # Verifica se o sistema está sendo usado
    if(sistema == True):
        clear_output(wait=True)
    # Seleciona os 10 primeiros
    top = (df.groupby(['Centro de Distribuição','Destino'])['Distância (km)']
             .mean().sort_values(ascending=False).head(10))
    # Seta o gráfico
    fig, ax = plt.subplots()
    # Escolhe o tipo de gráfico
    top.plot(kind='barh', ax=ax)
    # Titulo
    ax.set_title('Top 10 Rotas mais longas por Distância Média')
    # horizontal
    ax.set_xlabel('Distância (km)')
    # Ajusta o layout para evitar sobreposição
    plt.tight_layout()
    # Exibe o gráfico
    plt.show()

analise_top_distancias(df_entregas)

In [ ]:
def analise_tempo_medio_viagem(df, sistema=False):
    # Faz um GroupBy e seleciona os 10 primeiros
    grp = (df.groupby(['Centro de Distribuição','Destino'])['Prazo de Entrega (dias)']
             .mean().sort_values(ascending=False).head(10))
    # Seta o gráfico
    fig, ax = plt.subplots()
    grp.plot(kind='barh', ax=ax)
    # Titulo
    ax.set_title('Top 10 Rotas com maior Tempo Médio')
    # Horizontal
    ax.set_xlabel('Tempo (Dias)')
    # Ajusta o layout para evitar sobreposição
    plt.tight_layout()
    # Exibe o gráfico
    plt.show()
    # Verifica se o sistema está sendo usado

analise_tempo_medio_viagem(df_entregas)

In [ ]:
def analise_custos(df, sistema=False):
    # Verifica se o sistema está sendo usado
    # Organiza o DF pela linha especifica
    fig, ax = plt.subplots(figsize=(8, 5))
    # Faz um plot com base nessa coluna
    ax.hist(df['Custo por Entrega (R$)'].dropna(), bins=20, color='skyblue', edgecolor='black')
    # Titulo
    ax.set_title('Histograma de Custos por Entrega')
    #configura os eixos x e y
    ax.set_xlabel('Custo por Entrega (R$)')
    ax.set_ylabel('Frequência')
    # Exibe o gráfico
    
    plt.show()
    # Verifica se o sistema está sendo usado

analise_custos(df_entregas)
